# INPUTS

Example extractable Tables stored in Hive for distance based criteria

In [1]:
bike_parking = {'location 1': (37.7606289177, -122.410647009), 
                'location 2': (37.7855355791102, -122.411302813025),
                'location 3': (37.7759676911831, -122.441396661871),
                'location 4': (37.7518243814, -122.426627114),
                'location 5': (37.75182438, -122.4266271)}

off_street_parking =      {'location 1': ["Private", 13, (37.7606289177, -122.410647009)], 
                'location 2': ["Private", 15, (37.7855355791102, -122.411302813025)],
                'location 3': ["Public", 18, (37.7759676911831, -122.441396661871)],
                'location 4': ["Private", 27, (37.7518243814, -122.426627114)],
                'location 5': ["Private", 60, (37.75182438, -122.4266271)]}

Example User Input

In [2]:
distance_to_bike_parking = "Short"      # Short, Medium, Long
distance_to_bike_share = "Short"        # Short, Medium, Long
density_of_offstreet_parking = "Low"   # Low, Medium, High Density within 500m
density_of_SFPD_Incidents = "Low"       # Low, Medium, High Density in 2016
density_of_trees_100m = "High"          # Low, Medium, High Density within 500m

Example Geotag

In [3]:
geotag = (37.790788, -122.419036)

# Bike Parking Function (Short, Medium, Long Distance Style Evaluation)

In [4]:
# Adapted from http://www.codecodex.com/wiki/Calculate_distance_between_two_points_on_a_globe#Python
import math
def points2distance(start,  end):  
    """ 
    Calculate distance (in kilometers) between two points given as (long, latt) pairs 
    based on Haversine formula (http://en.wikipedia.org/wiki/Haversine_formula). 
    """  
    end_latt = math.radians(end[0])
    end_long = math.radians(end[1])
    start_latt = math.radians(start[0])
    start_long = math.radians(start[1])
        
    d_latt = end_latt - start_latt  
    d_long = end_long - start_long  
    a = math.sin(d_latt/2)**2 + math.cos(start_latt) * math.cos(end_latt) * math.sin(d_long/2)**2  
    c = 2 * math.asin(math.sqrt(a))  
    return 6371 * c  

In [5]:
def close_to_bike_parking(geotag, bike_parking_locations):

    min_dist = None
    near_bike = False
    bike_dist = "N/A"
    bike = ""
    MAX_BIKE_DIST = 200 # kilometers

    for spot, coords in bike_parking_locations.items():
        dist = points2distance(coords,geotag)
        if (min_dist is None or dist < min_dist) and dist < MAX_BIKE_DIST:
            bike = spot
            near_bike = True
            min_dist = dist
    return near_bike, bike, min_dist
        
        
        
    
    

In [6]:
# Test
close_to_bike_parking(geotag,bike_parking)

(True, 'location 2', 0.8960481671803393)

# Vehicle Parking Function (Low, Medium, High Density Evaluation)

In [7]:
def parking_density(geotag, parking_locations):

    public_parking_count = 0
    private_parking_count = 0
    MAX_PARK_DIST = 3.5 # kilometers
    low_density_threshold = 20
    med_density_threshold = 100
    
    # Loop through Parking Locations and sum total available spots (within Max Distance of geotag) for both Private/Public
    for name, info in parking_locations.items():
        dist = points2distance(info[2],geotag)
        if dist < MAX_PARK_DIST and info[0] == "Public":
            public_parking_count += info[1] # Assuming the Parking lot info has # of spots in the raw data
        elif dist < MAX_PARK_DIST and info[0] == "Private":
            private_parking_count += info[1]
        else:
            continue
    
    
    # Characterize Public Parking Density
    public_parking_density = None
    if public_parking_count > 0 and public_parking_count <= low_density_threshold:
        public_parking_density = "Low Public Parking Density"
    elif public_parking_count > low_density_threshold and public_parking_count <= med_density_threshold:
        public_parking_density = "Medium Public Parking Density"
    elif public_parking_count > med_density_threshold:
        public_parking_density = "High Public Parking Density"
    
    # Characterize Private Parking Density
    private_parking_density = None
    if private_parking_count > 0 and private_parking_count <= low_density_threshold:
        private_parking_density = "Low Private Parking Density"
    elif private_parking_count > low_density_threshold and private_parking_count <= med_density_threshold:
        private_parking_density = "Medium Private Parking Density"
    elif private_parking_count > med_density_threshold:
        private_parking_density = "High Private Parking Density"
        
        
    return (public_parking_density, public_parking_count, private_parking_density, private_parking_count)

In [8]:
# Test
parking_density(geotag,off_street_parking)

('Low Public Parking Density', 18, 'Medium Private Parking Density', 28)